In [1]:
include("../ToolBox/ToolBox.jl")
import JLD
import FileIO
using .ToolBox
using .Geometry
using Colors

In [2]:
dem_path = "/Users/simon/Data/DEM/houston_small.jld";
dem_nan = -25;

co_folder = "/Users/simon/Data/Sentinel/houston/S1B_IW_SLC__1SDV_20170830T122203_20170830T122233_007169_00CA2C_C92C.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/houston/S1A_IW_SLC__1SDV_20170824T122248_20170824T122318_018065_01E54E_5C27.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/houston/S1B_IW_SLC__1SDV_20170818T122205_20170818T122235_006994_00C525_0F49.SAFE"
product_folders = [co_folder,pre1_folder,pre2_folder];

In [3]:
target = 2

2

In [4]:
    n_coherence = length(product_folders)-1


2

In [18]:
master_idx = ones(n_coherence)*target
slave_idx = collect(1:length(product_folders))
slave_idx = slave[slave .!= target]

2-element Array{Int64,1}:
 1
 3

In [ ]:
lut_array = Array{Any}(undef, n_coherence)

In [2]:
function preprocess3_v1(product_folders, to_one, target, master_view, dem_path, subswath, result_folder, 
    dem_nan,kernel = ones(4,14))
    
    folder_idx = collect(1:length(product_folders))
    n_coherence = length(product_folders)-1
    
    if to_one
        master_idx = ones(n_coherence)*target
        slave_idx = collect(1:length(product_folders))
        slave_idx = folder_idx[folder_idx .!= target]
    else
        master_idx = folder_idx[2:end]
        slave_idx = folder_idx[1:end-1]
    end
    
    lut_array = Array{Any}(undef, n_coherence)
    lut_array .= Missing
    
    dem = 0
    
    precise_orbit = 0
    master_data = 0
    lines = 0
    samples = 0
    
    stride_line = floor(Int,size(kernel)[1]/2)
    stride_sample = floor(Int,size(kernel)[2]/2)
    
    pod_paths = [Load.pod_path(folder) for folder in product_folders]
    
    for polarization in ["VV","VH"]
        
        println(polarization)
        file_paths = [Load.slc_paths(folder, polarization, subswath) for folder in product_folders]
        meta = [Load.slc_meta(path[2]) for path in file_paths]
        calibration = [Load.slc_calibration(file_paths[i][3],meta[i]["t_0"]) 
                                    for i in 1:length(product_folders)];
        # load pod
        if precise_orbit == 0
            println("load POD")
            precise_orbit = [Load.precise_orbit(pod_paths[i],meta[i]["t_0"]) 
                                    for i in 1:length(product_folders)]
        end
        
        # load DEM
        if dem == 0
            println("load DEM")
            pad = 0.1
            dem = JLD.load(dem_path);
            footprint = SlcUtil.footprint(meta[2], master_view)
            latlon_window = ((minimum(footprint[1]), maximum(footprint[1])), (minimum(footprint[2]), maximum(footprint[2])))
            idx1 =(latlon_window[1][1] - pad) .<dem["lat"].< (latlon_window[1][2] + pad)
            idx2 =(latlon_window[2][1] - pad) .<dem["lon"].< (latlon_window[2][2] + pad)
            dem = (dem["lat"][idx1], dem["lon"][idx2], dem["height"][idx1,idx2]);
            dem[3][dem[3].== -32768] .= dem_nan; 
        end
        
        for i in 1:n_coherence
            println("i: ",i)
            index = [master_idx[i], slave_idx[i]]
            
            # coreg_slave data
            if lut_array[i] == Missing
                println("coreg no lut")
                slave_data,flat,lut_array[i] =  coregister_slave(master_view,file_paths[index[2]][1],
                                                           meta[index],precise_orbit[index],dem)
                lut_path = joinpath(result_folder,string(index[1])*string(index[2])*"_lut.jld")
                JLD.save(lut_path, "data", lut_array[i])
                
            else
                println("coreg with lut")
                slave_data,flat =  coregister_slave(master_view,file_paths[index[2]][1],
                                                        meta[index],precise_orbit[index],dem,lut_array[i])
            end

            slave_data,mosaic_view = SlcUtil.mosaic(slave_data,master_view,meta[index[1]]);
            flat,mosaic_view = SlcUtil.mosaic(flat,master_view,meta[index[1]]);
            slave_data = SlcUtil.calibrate_slave_data(slave_data, mosaic_view,lut1, calibration[index[2]]);
            
            master_data = Load.slc_data(file_paths[index[1]][1],master_view);
            master_data,mosaic_view = SlcUtil.mosaic(master_data,master_view,meta[index[1]]);
            master_data = SlcUtil.calibrate_data(master_data, Misc.flatten(mosaic_view...)..., calibration[index[1]]);

            # Compute coherence
            complex_coherence, master_intensity, slave_intensity, lines, samples = SlcUtil.complex_coherence(
                                                                master_data, slave_data, flat, kernel, mosaic_view);
            # subsample 
            complex_coherence = complex_coherence[1:stride_line:end,1:stride_sample:end]
            slave_intensity = slave_intensity[1:stride_line:end,1:stride_sample:end]
            
            # Transform to target cordinates..............................
            # Fuck this shit will not work.
            # different master views for each coherence 
            # make to end only and then update old to transform to end image 
            # To general.
            
            # save 
            slave_path = joinpath(result_folder,string(index[2])*polarization*"_itens.jld")
            master_path = joinpath(result_folder,string(index[1])*polarization*"_itens.jld")
            coherence_path = joinpath(result_folder,string(index[1])*string(index[2])*
                                                            polarization*"_coher.jld")
            println("Save")
            JLD.save(slave_path, "data", slave_intensity)
            JLD.save(coherence_path, "data", complex_coherence)
            if i == 1
                println("Save master intens")
                master_intensity = master_intensity[1:stride_line:end,1:stride_sample:end]
                JLD.save(master_path, "data", master_intensity)
            end
            
        end
        
    end
    
    println("Save AUX info")
    lines = lines[1:stride_line:end]
    samples = samples[1:stride_sample:end]
    
    # save lines and samples 
    line_sample_path = joinpath(result_folder,"line_sample.jld")
    JLD.save(line_sample_path, "lines", lines, "samples", samples)
    
    # Save heights
    heights = Misc.interp_grid(lut1["master_line"] ,lut1["master_sample"],
        reshape(lut1["heights"],(length(lut1["master_line"]),length(lut1["master_sample"])))
        ,lines, samples);
    
    heights_path = joinpath(result_folder,"heights.jld")
    JLD.save(heights_path, "data", heights)
    
    return 1
    
end

preprocess3_v1 (generic function with 2 methods)

# Huston 

In [3]:
dem_path = "/Users/simon/Data/DEM/houston_small.jld";
dem_nan = -25;

co_folder = "/Users/simon/Data/Sentinel/houston/S1B_IW_SLC__1SDV_20170830T122203_20170830T122233_007169_00CA2C_C92C.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/houston/S1A_IW_SLC__1SDV_20170824T122248_20170824T122318_018065_01E54E_5C27.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/houston/S1B_IW_SLC__1SDV_20170818T122205_20170818T122235_006994_00C525_0F49.SAFE"
product_folders = [co_folder,pre1_folder,pre2_folder];

### River

In [4]:
result_folder = "/Users/simon/Data/Sentinel/houston/river_BAB"
master_B_view = [7600:9500,7000:15000];

preprocess3_v1(product_folders,master_B_view,dem_path,1, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

### city center

In [4]:
result_folder = "/Users/simon/Data/Sentinel/houston/city_center_BAB"
master_B_view = [7600:9500,11000:19000];

preprocess3_v1(product_folders,master_B_view,dem_path,1, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

### Resovior

In [5]:
result_folder = "/Users/simon/Data/Sentinel/houston/resovoir_BAB"
master_B_view = [7000:9500,500:5000];

preprocess3_v1(product_folders,master_B_view,dem_path,2, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

## Vejle

In [6]:
dem_path = "/Users/simon/Data/DEM/denmark.jld";
dem_nan = 40;

In [7]:
co_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20191011T170932_20191011T170959_029414_035865_06AB.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190929T170932_20190929T170959_029239_035263_C3F9.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190917T170931_20190917T170958_029064_034C5E_2484.SAFE"

result_folder = "/Users/simon/Data/Sentinel/vejle_oktober/factory_AAA"

master_A_view = [8000:10000,1200:6000];
product_folders = [co_folder,pre1_folder,pre2_folder];

preprocess3_v1(product_folders,master_A_view,dem_path,3, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1

In [8]:
co_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20191011T170932_20191011T170959_029414_035865_06AB.SAFE"
pre1_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1B_IW_SLC__1SDV_20191005T170849_20191005T170916_018343_0228DC_F565.SAFE"
pre2_folder = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190929T170932_20190929T170959_029239_035263_C3F9.SAFE"

result_folder = "/Users/simon/Data/Sentinel/vejle_oktober/factory_ABA"

product_folders = [co_folder,pre1_folder,pre2_folder];

master_B_view = [9500:11500,1200:6000];

preprocess3_v1(product_folders,master_B_view,dem_path,3, result_folder,dem_nan)

VV
load POD
load DEM
i: 1
coreg no lut
Save
Save master intens
i: 2
coreg no lut
Save
VH
i: 1
coreg with lut
Save
Save master intens
i: 2
coreg with lut
Save
Save AUX info


1